# Differential expression analysis

## Install and import dependencies

In [ ]:
source("https://bioconductor.org/biocLite.R")
biocLite()

In [ ]:
biocLite(c("limma", "Glimma", "edgeR", "clusterProfiler"))

In [ ]:
library(limma)
library(Glimma)
library(edgeR)
library(RColorBrewer)
library(scales)
library(heatmaply)
library(clusterProfiler)
library(grDevices)
library(ggplot2)
library(ggrepel)

In [ ]:
getwd()

## Example analysis
https://www.bioconductor.org/help/workflows/RNAseq123/

In [ ]:
biocLite("Mus.musculus")
library(Mus.musculus)

### Load input files

In [ ]:
url <- "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE63310&format=file"
utils::download.file(url, destfile="GSE63310_RAW.tar", mode="wb") 
utils::untar("GSE63310_RAW.tar", exdir = ".")
files <- c("GSM1545535_10_6_5_11.txt", "GSM1545536_9_6_5_11.txt", "GSM1545538_purep53.txt",
  "GSM1545539_JMS8-2.txt", "GSM1545540_JMS8-3.txt", "GSM1545541_JMS8-4.txt",
  "GSM1545542_JMS8-5.txt", "GSM1545544_JMS9-P7c.txt", "GSM1545545_JMS9-P8c.txt")
for(i in paste(files, ".gz", sep=""))
  R.utils::gunzip(i, overwrite=TRUE)

In [ ]:
files <- c("GSM1545535_10_6_5_11.txt", "GSM1545536_9_6_5_11.txt", 
   "GSM1545538_purep53.txt", "GSM1545539_JMS8-2.txt", 
   "GSM1545540_JMS8-3.txt", "GSM1545541_JMS8-4.txt", 
   "GSM1545542_JMS8-5.txt", "GSM1545544_JMS9-P7c.txt", 
   "GSM1545545_JMS9-P8c.txt")
# Show data format
read.delim(files[1], nrow=5)

In [ ]:
# Directly construct matrix of gene count per condition
x <- readDGE(files, columns=c(1,3))
class(x)
dim(x)

In [ ]:
# Input sample annotation
samplenames <- substring(colnames(x), 12, nchar(colnames(x)))
colnames(x) <- samplenames
group <- as.factor(c("LP", "ML", "Basal", "Basal", "ML", "LP", 
                     "Basal", "ML", "LP"))
x$samples$group <- group
lane <- as.factor(rep(c("L004","L006","L008"), c(3,4,2)))
x$samples$lane <- lane
x$samples

In [ ]:
# Input gene annotation
geneid <- rownames(x)
genes <- select(Mus.musculus, keys=geneid, columns=c("SYMBOL", "TXCHROM"), 
                keytype="ENTREZID")
# Choose to keep only the first chromosome position of each gene
genes <- genes[!duplicated(genes$ENTREZID),]
head(genes)

x$genes <- genes

### Transform and filter data on abundance
CPM (counts per million) are just normalized by abundance in sample
Allows to compare between samples but not across genes (no normalization on gene length)

In [ ]:
# head(10**6 * x$counts[,3] / colSums(x$counts)[3])
xCPM = cpm(x)
xLogCPM = cpm(x, log = T)
# NB: prior count of 0.25

In [ ]:
table(rowSums(x$counts==0)==9)

In [ ]:
keep.exprs <- rowSums(xCPM>1)>=3
x <- x[keep.exprs,, keep.lib.sizes=FALSE]
dim(x)

In [ ]:
xLogCPMfiltered = cpm(x, log = T)

nsamples <- ncol(x)
col <- brewer.pal(nsamples, "Paired")

par(mfrow=c(1,2))

plot(density(xLogCPMfiltered[,1]), col=col[1], lwd=2, ylim=c(0,0.21), las=2, 
     main="", xlab="")
title(main="A. Filtered data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
 den <- density(xLogCPMfiltered[,i])
 lines(den$x, den$y, col=col[i], lwd=2)
}
legend("topright", samplenames, text.col=col, bty="n")

plot(density(xLogCPM[,1]), col=col[1], lwd=2, ylim=c(0,0.21), las=2, 
     main="", xlab="")
title(main="B. Raw data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
   den <- density(xLogCPM[,i])
   lines(den$x, den$y, col=col[i], lwd=2)
}
legend("topright", samplenames, text.col=col, bty="n")

In [ ]:
x <- calcNormFactors(x, method = "TMM")
x$samples

In [ ]:
par(bg="white")
plotMDS(x) # Three distinct groups (LP/ML/Basal)

In [ ]:
design <- model.matrix(~0+group+lane)
colnames(design) <- gsub("group", "", colnames(design))
design

In [ ]:
contr.matrix <- makeContrasts(
   BasalvsLP = Basal-LP, 
   BasalvsML = Basal - ML, 
   LPvsML = LP - ML, 
   levels = colnames(design))
contr.matrix

In [ ]:
par(mfrow=c(1,2), bg="white")
v <- voom(x, design, plot=TRUE)
vfit <- lmFit(v, design)
vfit <- contrasts.fit(vfit, contrasts=contr.matrix)
efit <- eBayes(vfit)
plotSA(efit, main="Final model")

In [ ]:
v

In [ ]:
summary(decideTests(efit))

In [ ]:
tfit <- treat(vfit, lfc=1)
dt <- decideTests(tfit)
summary(dt)

In [ ]:
de.common <- which(dt[,1]!=0 & dt[,2]!=0)
length(de.common)

In [ ]:
head(tfit$genes$SYMBOL[de.common], n=20)

In [ ]:
par(bg = "white")
vennDiagram(dt[,1:3], circle.col=c("turquoise", "salmon"))

In [ ]:
basal.vs.lp <- topTreat(tfit, coef=1, n=Inf)
basal.vs.ml <- topTreat(tfit, coef=2, n=Inf)
head(basal.vs.lp)

In [ ]:
par(bg="white")
plotMD(tfit, column=1, status=dt[,1], main=colnames(tfit)[1], 
       xlim=c(-8,13))

In [ ]:
basal.vs.lp.topgenes <- basal.vs.lp$ENTREZID[1:100]
i <- which(v$genes$ENTREZID %in% basal.vs.lp.topgenes)
colPal <- colorRampPalette(c("#4A91C4","white","#F09F4E"))

In [ ]:
heatmaply(v$E[i,], labCol=paste(colnames(v), group), labRow=v$genes$SYMBOL[i], branches_lwd = 0.25, col = colPal, 
    cexCol = 0.8, cexRow = 0.1, margins = c(80,0,5,0))

In [ ]:
setwd("/Users/lvulliard/OneShotProject/RecArg1/counts")

## RecArg1 - Counts with multi-mapping reads

In [ ]:
countMat = matrix(nrow=26301, ncol=28)
colnames(countMat) <- list.files()
for (i in 1:28) {
    load(colnames(countMat)[i])
    countMat[,i] <- fcMulti$counts
}
rownames(countMat) <- rownames(fcMulti$counts)
# Four replicates of seven different conditions
conditionList = c(5,6,7,1,1,2,2,3,3,4,4,5,1,7,5,2,3,4,6,7,7,1,2,4,5,6,3,6)
# If s06 is Control RANKL rescue and s08 is Control rescue
# conditionList = c(5,6,7,1,1,2,2,3,3,4,4,5,1,7,5,2,3,4,6,7,7,1,2,4,5,6,6,3)

In [ ]:
# Convert to DGE object
dgeData = DGEList(counts = countMat, genes = rownames(countMat), group = conditionList)

In [ ]:
dim(dgeData)

In [ ]:
names(dgeData)

In [ ]:
# Counts equal to 0 for all but 3 samples at most
table(rowSums(dgeData$counts==0)>=25)
# Counts equal to 0 for all sample
table(rowSums(dgeData$counts==0)==28)

In [ ]:
dgeDataLogCPMUnfiltered = cpm(dgeData, log = T)

In [ ]:
table(rowSums(dgeDataLogCPMUnfiltered > 0) > 3)

In [ ]:
head(dgeData$counts)

In [ ]:
dgeData = dgeData[rowSums(dgeDataLogCPMUnfiltered > 0) > 3,, keep.lib.sizes=FALSE]

In [ ]:
head(dgeData$counts)

In [ ]:
length(rownames(dgeData))

3862 genes are never expressed, 6090 are not expressed in more than 3 conditions, so not even in all replicates of a single condition.  
We keep the 12952 genes with a logCPM value of one or higher in 4 or more samples.

In [ ]:
# Input gene annotation
genes <- select(Mus.musculus, keys=rownames(dgeData), columns=c("SYMBOL", "TXCHROM"), 
                keytype="ENTREZID")
# Choose to keep only the first chromosome position of each gene
genes <- genes[!duplicated(genes$ENTREZID),]
head(genes)

dgeData$genes <- genes

In [ ]:
# Input sample annotation
dgeData$samples$aMEM = c(TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE)[conditionList]
dgeData$samples$aMEMwoLArg = c(FALSE,TRUE,TRUE,FALSE,TRUE,TRUE,FALSE)[conditionList]
dgeData$samples$LArg = c(FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE)[conditionList]
dgeData$samples$recArg = c(FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE)[conditionList]
dgeData$samples$RANKL = c(FALSE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE)[conditionList]
# dgeData$samples$mouse = c()
dgeData$samples

In [ ]:
dgeDataLogCPM = cpm(dgeData, log = T)
# NB: prior count of 0.25

nsamples <- ncol(x)
col <- brewer.pal(nsamples, "Paired")

par(mfrow=c(1,2), bg="white")

plot(density(dgeDataLogCPM[,1]), col=col[1], lwd=2, ylim=c(0,0.21), las=2, 
     main="", xlab="")
title(main="A. Filtered data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
 den <- density(dgeDataLogCPM[,i])
 lines(den$x, den$y, col=col[i], lwd=2)
}
legend("topright", rownames(dgeData$samples), text.col=col, bty="n")

plot(density(dgeDataLogCPMUnfiltered[,1]), col=col[1], lwd=2, ylim=c(0,0.21), las=2, 
     main="", xlab="")
title(main="B. Raw data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
   den <- density(dgeDataLogCPMUnfiltered[,i])
   lines(den$x, den$y, col=col[i], lwd=2)
}
legend("topright", rownames(dgeData$samples), text.col=col, bty="n")

In [ ]:
pdf("../filteringRNA.pdf")

par(family = "sans", mfrow=c(1,2), bg="white")

plot(density(dgeDataLogCPMUnfiltered[,1]), col=col[1], lwd=2, ylim=c(0,0.21), las=2, 
     main="", xlab="")
title(main="Raw data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
   den <- density(dgeDataLogCPMUnfiltered[,i])
   lines(den$x, den$y, col=col[i], lwd=2)
}

plot(density(dgeDataLogCPM[,1]), col=col[1], lwd=2, ylim=c(0,0.21), las=2, 
     main="", xlab="")
title(main="Filtered data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
 den <- density(dgeDataLogCPM[,i])
 lines(den$x, den$y, col=col[i], lwd=2)
}

dev.off()

In [ ]:
dgeData <- calcNormFactors(dgeData, method = "TMM")
dgeData$samples

In [ ]:
sampleLabels = paste0("s", 1:28, "_", conditionList)
heatmaply(cor(dgeData$counts),symm=T, labRow = sampleLabels, labCol = sampleLabels, branches_lwd = 0.25, margins = c(50,50,5,0), cexRow = 0.6)

In [ ]:
par(bg="white")
colpal = brewer.pal(7, "Set2")
plotMDS(dgeData, labels = sampleLabels, col = colpal[as.numeric(dgeData$samples$group)])

In [ ]:
pdf("../mdsMultiRNA.pdf")
par(family = "sans")
plotMDS(dgeData, labels = sampleLabels, col = colpal[as.numeric(dgeData$samples$group)])
dev.off()

In [ ]:
# Which mice are the samples coming from?
mouse = c(1,1,1,2,1,2,1,2,1,2,1,2,4,3,4,4,4,4,4,4,2,3,3,3,3,3,2,3)
table(mouse)

In [ ]:
design <- with(data.frame(dgeData$samples), 
#    model.matrix(~0+aMEM+LArg+recArg+RANKL))
    model.matrix(~0+group))
design

In [ ]:
contr.matrix <- makeContrasts(
                        g1v2 = group1 - group2,
                        g1v3 = group1 - group3,
                        g1v4 = group1 - group4,
                        g2v3 = group2 - group3,
                        g2v5 = group2 - group5,
                        g3v6 = group3 - group6,
                        g4v5 = group4 - group5,
                        g4v6 = group4 - group6,
                        g4v7 = group4 - group7,
                        g5v7 = group5 - group7,
                        rankl = group6 + group5 + group4 - group3 - group2 - group1,
                        arg = group1 + group4 - group5 - group2,
                        rescue = group1 + group4 - group6 - group3,
                        levels = colnames(design))
contr.matrix

In [ ]:
par(mfrow=c(1,2), bg="white")
#Correct for heteroscedasticity 
v <- voom(dgeData, design, plot=TRUE)
# Linear model
vfit <- lmFit(v, design)
vfit <- contrasts.fit(vfit, contrasts=contr.matrix)
# Moderated t-statistics, moderated F-statistic, and log-odds of differential expression by empirical Bayes 
# moderation of the standard errors towards a common value 
efit <- eBayes(vfit)
plotSA(efit, main="Final model")

In [ ]:
# How many genes are differentially expressed overall
summary(decideTests(efit), lfc=1)

In [ ]:
# Same as eBayes but only for genes that have a log2 fold-change higher than 1
tfit <- treat(vfit, lfc=1)
# FDR = 5%
dt <- decideTests(tfit)
summary(dt)

In [ ]:
# Genes diff. expr. in g1v2 and g1v3
de.common <- which(dt[,1]!=0 & dt[,2]!=0)
length(de.common)

In [ ]:
tfit$genes$SYMBOL[de.common]

In [ ]:
pdf("../vennMultiRNA.pdf")
par(family = "sans")
vennDiagram(dt[,c(7,9,10)], circle.col=c("turquoise", "salmon", "forestgreen"))
dev.off()

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(7,9,10)], circle.col=c("turquoise", "salmon", "forestgreen"))

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(7,9)], circle.col=c("turquoise", "salmon"))

Adding recArg1 modified more gene expressions than lack of arginine.  
Most of the genes perturbed by lack of arginine are also perturbed by recArg1.  
Looking at the difference between arg$^-$ and recArg1$^+$ samples doesn't show additional information,

### Effect of RANKL

In [ ]:
pdf("../vennMultiRanklRNA.pdf")
par(family = "sans")
vennDiagram(dt[,c(3,5,6,11)], circle.col=c("turquoise", "salmon", "forestgreen", "tomato2"))
dev.off()

In [ ]:
pdf("../vennMultiRanklRNA2.pdf")
par(family = "sans")
vennDiagram(dt[,c(3,5,6)], circle.col=c("turquoise", "salmon", "forestgreen"))
dev.off()

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(3,5,6,11)], circle.col=c("turquoise", "salmon", "forestgreen", "tomato2"))

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(3,5,6)], circle.col=c("turquoise", "salmon"))

### Effect of arginin depletion

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(1,7,12)], circle.col=c("turquoise", "salmon"))

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(1,7)], circle.col=c("turquoise", "salmon"))

### Effect of arginin rescue

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(2,8,13)], circle.col=c("turquoise", "salmon"))

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(2,8)], circle.col=c("turquoise", "salmon"))

In [ ]:
top4v5 <- topTreat(tfit, coef=7, n=Inf)
top4v7 <- topTreat(tfit, coef=9, n=Inf)
head(top4v7)

In [ ]:
par(bg="white")
plotMD(tfit, column=9, status=dt[,9], main=colnames(tfit)[9], col = c("#4A91C4","#F09F4E"), bg.col = "grey", legend = F)

In [ ]:
pdf("../MDMultiRNA.pdf")
par(family = "sans")
plotMD(tfit, column=9, status=dt[,9], main=colnames(tfit)[9], col = c("#4A91C4","#F09F4E"), bg.col = "grey", legend = F)
dev.off()

In [ ]:
for (i in head(top4v7$SYMBOL, 40)){ cat(paste0(i, "\n")) }

In [ ]:
top4v7.topgenes <- top4v7$ENTREZID[1:40]
i <- which(v$genes$ENTREZID %in% top4v7.topgenes)
colPal <- colorRampPalette(c("#4A91C4","white","#F09F4E"))
heatmaply(v$E[i,], labCol=sampleLabels, labRow=v$genes$SYMBOL[i], branches_lwd = 0.25, col = colPal, file = "../heatmapMultiRNA.pdf",
    cexCol = 0.8, cexRow = 0.1, margins = c(80,0,5,0), ColSideColors=dgeData$samples$group, plot_method = "plotly")

In [ ]:
i <- which(v$genes$ENTREZID %in% top4v7$ENTREZID[1:4])
heatmaply(v$E[i,], labCol=sampleLabels, labRow=v$genes$SYMBOL[i], branches_lwd = 0.25, col = colPal, 
    cexCol = 0.8, cexRow = 0.1, margins = c(80,0,5,0))

top4v7[1:4,]

In [ ]:
dt.multi = dt

### Differences between recArg1 and arginine depletion in presence of RANKL
Conditions 7 and 5

In [ ]:
suppressWarnings(which(apply(dt.multi[,c(7,9,10)], 1, all)))
dt.multi[suppressWarnings(which(apply(dt.multi[,c(7,9,10)], 1, all))),c(7,9,10)]

Prostate transmembrane protein, androgen induced 1 (ENTREZ gene 65112) is overexpressed in arginine depletion compared to osteoclasts and recArg1, whereas it's underexpressed when comparing recArg1 to osteoclasts.

In [ ]:
top5v7 <- topTreat(tfit, coef=10, n=Inf)
head(top5v7)

In [ ]:
par(bg="white")
plotMD(tfit, column=10, status=dt[,10], main=colnames(tfit)[10], col = c("#4A91C4","#F09F4E"), bg.col = "grey", legend = F)

In [ ]:
pdf("../MDMultiRecArgVsDepl.pdf")
par(family = "sans")
plotMD(tfit, column=10, status=dt[,10], main=colnames(tfit)[10], col = c("#4A91C4","#F09F4E"), bg.col = "grey", legend = F)
dev.off()

In [ ]:
for (i in head(top5v7$SYMBOL, 40)){ cat(paste0(i, "\n")) }

In [ ]:
top5v7.topgenes <- top5v7$ENTREZID[1:40]
i <- which(v$genes$ENTREZID %in% top5v7.topgenes)
colPal <- colorRampPalette(c("#4A91C4","white","#F09F4E"))
heatmaply(v$E[i,], labCol=sampleLabels, labRow=v$genes$SYMBOL[i], branches_lwd = 0.25, col = colPal, file = "../heatmapMultiRecArgVsDepl.pdf",
    cexCol = 0.8, cexRow = 0.1, margins = c(80,0,5,0), ColSideColors=dgeData$samples$group, plot_method = "plotly")

### Export expression for OmicsIntegrator

In [ ]:
write.table(top4v7[(top4v7$ENTREZID %in% names(which(dt[,9] != 0))),c(2,4)], file = "../OmicsIntegratorRun/recArg1_expr.tsv", 
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE) 

In [ ]:
write.table(top5v7[(top5v7$ENTREZID %in% names(which(dt[,10] != 0))),c(2,4)], file = "../OmicsIntegratorRun/deplOrRec_expr.tsv", 
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE) 

In [ ]:
upDeplGenes1 = names(which((dt.multi[,3] == 1) & (dt.multi[,6] == 1) & (dt.multi[,5] == 0)))
downDeplGenes1 = names(which((dt.multi[,3] == -1) & (dt.multi[,6] == -1) & (dt.multi[,5] == 0)))
notUpDeplGenes1 = names(which((dt.multi[,3] == 0) & (dt.multi[,6] == 0) & (dt.multi[,5] == 1)))
notDownDeplGenes1 = names(which((dt.multi[,3] == 0) & (dt.multi[,6] == 0) & (dt.multi[,5] == -1)))

In [ ]:
deplGenes = union(union(upDeplGenes1, downDeplGenes1), union(notUpDeplGenes1, notDownDeplGenes1))

In [ ]:
deplGenes

In [ ]:
top3v6 <- topTreat(tfit, coef=6, n=Inf)
top2v5 <- topTreat(tfit, coef=5, n=Inf)

In [ ]:
lfcFc = unlist(sapply(deplGenes, function(i) log2(2**(top2v5[top2v5$ENTREZID == i,4])/2**(top3v6[top3v6$ENTREZID == i,4]))))

In [ ]:
lfcFc = abs(lfcFc)
write.table(data.frame(v$genes$SYMBOL[v$genes$ENTREZID %in% deplGenes[lfcFc > 0.5]], lfcFc[lfcFc > 0.5]), 
                       file = "../OmicsIntegratorRun/ranklDiff_expr.tsv", 
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE) 

## RecArg1 - Counts without multi-mapping reads

In [ ]:
countMat = matrix(nrow=26301, ncol=28)
colnames(countMat) <- list.files()
for (i in 1:28) {
    load(colnames(countMat)[i])
    countMat[,i] <- fcStrict$counts
}
rownames(countMat) <- rownames(fcStrict$counts)
# Four replicates of seven different conditions
conditionList = c(5,6,7,1,1,2,2,3,3,4,4,5,1,7,5,2,3,4,6,7,7,1,2,4,5,6,3,6)
# If s06 is Control RANKL rescue and s08 is Control rescue
# conditionList = c(5,6,7,1,1,2,2,3,3,4,4,5,1,7,5,2,3,4,6,7,7,1,2,4,5,6,6,3)

In [ ]:
# Convert to DGE object
dgeData = DGEList(counts = countMat, genes = rownames(countMat), group = conditionList)

In [ ]:
dim(dgeData)

In [ ]:
names(dgeData)

In [ ]:
# Counts equal to 0 for all but 3 samples at most
table(rowSums(dgeData$counts==0)>=25)
# Counts equal to 0 for all sample
table(rowSums(dgeData$counts==0)==28)

In [ ]:
dgeDataLogCPMUnfiltered = cpm(dgeData, log = T)

In [ ]:
table(rowSums(dgeDataLogCPMUnfiltered > 0) > 3)

In [ ]:
dgeData = dgeData[rowSums(dgeDataLogCPMUnfiltered > 0) > 3,, keep.lib.sizes=FALSE]
# dgeData = dgeData[rowSums(dgeData$counts==0)>=25,, keep.lib.sizes=FALSE]

5010 genes are never expressed, 7267 are not expressed in more than 3 conditions, so not even in all replicates of a single condition.
We keep the 12952 genes with a logCPM value of one or higher in 4 or more samples.

In [ ]:
# Input gene annotation
genes <- select(Mus.musculus, keys=rownames(dgeData), columns=c("SYMBOL", "TXCHROM"), 
                keytype="ENTREZID")
# Choose to keep only the first chromosome position of each gene
genes <- genes[!duplicated(genes$ENTREZID),]
head(genes)

dgeData$genes <- genes

In [ ]:
# Input sample annotation
dgeData$samples$aMEM = c(TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE)[conditionList]
dgeData$samples$aMEMwoLArg = c(FALSE,TRUE,TRUE,FALSE,TRUE,TRUE,FALSE)[conditionList]
dgeData$samples$LArg = c(FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE)[conditionList]
dgeData$samples$recArg = c(FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE)[conditionList]
dgeData$samples$RANKL = c(FALSE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE)[conditionList]
# dgeData$samples$mouse = c()
dgeData$samples

In [ ]:
dgeDataLogCPM = cpm(dgeData, log = T)
# NB: prior count of 0.25

nsamples <- ncol(x)
col <- brewer.pal(nsamples, "Paired")

par(mfrow=c(1,2), bg="white")

plot(density(dgeDataLogCPM[,1]), col=col[1], lwd=2, ylim=c(0,0.21), las=2, 
     main="", xlab="")
title(main="A. Filtered data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
 den <- density(dgeDataLogCPM[,i])
 lines(den$x, den$y, col=col[i], lwd=2)
}
legend("topright", rownames(dgeData$samples), text.col=col, bty="n")

plot(density(dgeDataLogCPMUnfiltered[,1]), col=col[1], lwd=2, ylim=c(0,0.21), las=2, 
     main="", xlab="")
title(main="B. Raw data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
   den <- density(dgeDataLogCPMUnfiltered[,i])
   lines(den$x, den$y, col=col[i], lwd=2)
}
legend("topright", rownames(dgeData$samples), text.col=col, bty="n")

In [ ]:
dgeData <- calcNormFactors(dgeData, method = "TMM")
dgeData$samples

In [ ]:
sampleLabels = paste0("s", 1:28, "_", conditionList)
heatmaply(cor(dgeData$counts),symm=T, labRow = sampleLabels, labCol = sampleLabels, branches_lwd = 0.25, margins = c(50,50,5,0), cexRow = 0.6)

In [ ]:
par(bg="white")
colpal = brewer.pal(7, "Set2")
plotMDS(dgeData, labels = sampleLabels, col = colpal[as.numeric(dgeData$samples$group)])

In [ ]:
with(data.frame(dgeData$samples), tvr <- aMEM)

In [ ]:
design <- with(data.frame(dgeData$samples), 
#    model.matrix(~0+aMEM+LArg+recArg+RANKL))
    model.matrix(~0+group))
design

In [ ]:
contr.matrix <- makeContrasts(
                        g1v2 = group1 - group2,
                        g1v3 = group1 - group3,
                        g1v4 = group1 - group4,
                        g2v3 = group2 - group3,
                        g2v5 = group2 - group5,
                        g3v6 = group3 - group6,
                        g4v5 = group4 - group5,
                        g4v6 = group4 - group6,
                        g4v7 = group4 - group7,
                        g5v7 = group5 - group7,
                        rankl = group6 + group5 + group4 - group3 - group2 - group1,
                        arg = group1 + group4 - group5 - group2,
                        rescue = group1 + group4 - group6 - group3,
                        levels = colnames(design))
contr.matrix

In [ ]:
par(mfrow=c(1,2), bg="white")
v <- voom(dgeData, design, plot=TRUE)
vfit <- lmFit(v, design)
vfit <- contrasts.fit(vfit, contrasts=contr.matrix)
efit <- eBayes(vfit)
plotSA(efit, main="Final model")

In [ ]:
summary(decideTests(efit))

In [ ]:
tfit <- treat(vfit, lfc=1)
dt <- decideTests(tfit)
summary(dt)

In [ ]:
de.common <- which(dt[,1]!=0 & dt[,2]!=0)
length(de.common)

In [ ]:
head(tfit$genes$SYMBOL[de.common], n=20)

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(7,9,10)], circle.col=c("turquoise", "salmon"))

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(7,9)], circle.col=c("turquoise", "salmon"))

Adding recArg1 modified more gene expressions than lack of arginine.  
Most of the genes perturbed by lack of arginine are also perturbed by recArg1.  
Looking at the difference between arg$^-$ and recArg1$^+$ samples doesn't show additional information,

### Effect of RANKL

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(3,5,6,11)], circle.col=c("turquoise", "salmon"))

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(3,5,6)], circle.col=c("turquoise", "salmon"))

### Effect of arginin depletion

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(1,7,12)], circle.col=c("turquoise", "salmon"))

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(1,7)], circle.col=c("turquoise", "salmon"))

### Effect of arginin rescue

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(2,8,13)], circle.col=c("turquoise", "salmon"))

In [ ]:
par(bg = "white")
vennDiagram(dt[,c(2,8)], circle.col=c("turquoise", "salmon"))

In [ ]:
top4v5 <- topTreat(tfit, coef=7, n=Inf)
top4v7 <- topTreat(tfit, coef=9, n=Inf)
head(top4v7)

In [ ]:
par(bg="white")
plotMD(tfit, column=9, status=dt[,9], main=colnames(tfit)[9], 
       xlim=c(-8,13))

In [ ]:
top4v7.topgenes <- top4v7$ENTREZID[1:100]
i <- which(v$genes$ENTREZID %in% top4v7.topgenes)
colPal <- colorRampPalette(c("#4A91C4","white","#F09F4E"))
heatmaply(v$E[i,], labCol=sampleLabels, labRow=v$genes$SYMBOL[i], branches_lwd = 0.25, col = colPal, 
    cexCol = 0.8, cexRow = 0.1, margins = c(80,0,5,0))

In [ ]:
top4v7[1:50,c(2,4)]

In [ ]:
dt.strict = dt

### Differences between recArg1 and arginine depletion in presence of RANKL
Conditions 7 and 5

In [ ]:
suppressWarnings(which(apply(dt.strict[,c(7,9,10)], 1, all)))
dt.strict[suppressWarnings(which(apply(dt.strict[,c(7,9,10)], 1, all))),c(7,9,10)]

Prostate transmembrane protein, androgen induced 1 (ENTREZ gene 65112) is overexpressed in arginine depletion compared to osteoclasts and recArg1, whereas it's underexpressed when comparing recArg1 to osteoclasts.

In [ ]:
top5v7 <- topTreat(tfit, coef=10, n=Inf)
head(top5v7)

In [ ]:
par(bg="white")
plotMD(tfit, column=10, status=dt[,10], main=colnames(tfit)[10], col = c("#4A91C4","#F09F4E"), bg.col = "grey", legend = F)

In [ ]:
pdf("../MDStrictRecArgVsDepl.pdf")
par(family = "sans")
plotMD(tfit, column=10, status=dt[,10], main=colnames(tfit)[10], col = c("#4A91C4","#F09F4E"), bg.col = "grey", legend = F)
dev.off()

In [ ]:
length(top5v7$SYMBOL)

In [ ]:
for (i in head(top5v7$SYMBOL, 40)){ cat(paste0(i, "\n")) }

In [ ]:
top5v7.topgenes <- top5v7$ENTREZID[1:40]
i <- which(v$genes$ENTREZID %in% top5v7.topgenes)
colPal <- colorRampPalette(c("#4A91C4","white","#F09F4E"))
heatmaply(v$E[i,], labCol=sampleLabels, labRow=v$genes$SYMBOL[i], branches_lwd = 0.25, col = colPal, file = "../heatmapStrictRecArgVsDepl.pdf",
    cexCol = 0.8, cexRow = 0.1, margins = c(80,0,5,0), ColSideColors=dgeData$samples$group, plot_method = "plotly")

## Compare results with and without multimapping

In [ ]:
dim(dt.multi)
dim(dt.strict)

In [ ]:
sum(dt.strict != 0)
sum(dt.multi != 0)

In [ ]:
commonGenes = intersect(rownames(dt.strict), rownames(dt.multi))
length(commonGenes)

In [ ]:
dt.multi.g4v7 = dt.multi[dt.multi[,9] != 0, 9]
dt.strict.g4v7 = dt.strict[dt.strict[,9] != 0, 9]
length(dt.multi.g4v7)
length(dt.strict.g4v7)

In [ ]:
commonGenes = intersect(names(dt.strict.g4v7), names(dt.multi.g4v7))
length(commonGenes)
length(commonGenes) / length(union(names(dt.strict.g4v7), names(dt.multi.g4v7)))

81% of the genes in the two lists are found in both.

In [ ]:
i <- which(v$genes$ENTREZID %in% commonGenes)
head(v$E[i,])

### Output lists of genes differentially expressed in both cases

In [ ]:
listDownRNA = intersect(names(which(dt.strict.g4v7 == 1)), names(which(dt.multi.g4v7 == 1)))
listUpRNA = intersect(names(which(dt.strict.g4v7 == -1)), names(which(dt.multi.g4v7 == -1)))
length(listDownRNA)
length(listUpRNA)

In [ ]:
i <- which(v$genes$ENTREZID %in% listUpRNA)
listUpRNA = v$genes$SYMBOL[i]
i <- which(v$genes$ENTREZID %in% listDownRNA)
listDownRNA = v$genes$SYMBOL[i]
save(file = "../RNAlists.diff", listUpRNA, listDownRNA )

In [ ]:
for (i in listDownRNA) {
    cat(paste0(i, "\n"))
}

In [ ]:
for (i in listUpRNA) {
    cat(paste0(i, "\n"))
}

### Differences between recArg1 and arginine depletion

In [ ]:
dt.multi.g5v7 = dt.multi[dt.multi[,10] != 0, 10]
dt.strict.g5v7 = dt.strict[dt.strict[,10] != 0, 10]
length(dt.multi.g5v7)
length(dt.strict.g5v7)

In [ ]:
commonGenes = intersect(names(dt.strict.g5v7), names(dt.multi.g5v7))
length(commonGenes)
length(commonGenes) / length(union(names(dt.strict.g5v7), names(dt.multi.g5v7)))

In [ ]:
listDownRecArgVsDepletion = intersect(names(which(dt.strict.g5v7 == 1)), names(which(dt.multi.g5v7 == 1)))
listUpRecArgVsDepletion = intersect(names(which(dt.strict.g5v7 == -1)), names(which(dt.multi.g5v7 == -1)))
length(listDownRecArgVsDepletion)
length(listUpRecArgVsDepletion)

i <- which(v$genes$ENTREZID %in% listUpRecArgVsDepletion)
listUpRecArgVsDepletion = v$genes$SYMBOL[i]
i <- which(v$genes$ENTREZID %in% listDownRecArgVsDepletion)
listDownRecArgVsDepletion = v$genes$SYMBOL[i]
save(file = "../RNAlistsRecArg1VsDepletion.diff", listUpRNA, listDownRNA )

In [ ]:
for (i in listDownRecArgVsDepletion) {
    cat(paste0(i, "\n"))
}

In [ ]:
i <- which(v$genes$SYMBOL %in% listDownRecArgVsDepletion)
head(v$E[i,which(dgeData$samples$group == 5)])
head(v$E[i,which(dgeData$samples$group == 7)])

Corresponding [Enrichr](http://amp.pharm.mssm.edu/Enrichr/enrich?dataset=3uzqw)

In [ ]:
for (i in listUpRecArgVsDepletion) {
    cat(paste0(i, "\n"))
}

### Differences between RANKL effect in presence and absence of arginine
Intersection between comparisons 6 (3/6) and 3 (1/4) against comparison 5 (2/5)

In [ ]:
length(notUpDeplGenes1)
length(notDownDeplGenes1)
length(upDeplGenes1)
length(downDeplGenes1)
# We find back the 217 genes differentially regulated in presence of arginine when adding RANKL and the 14 in absence of arginine, 
# as shown on corresponding Venn diagram

In [ ]:
upDeplGenes2 = names(which((dt.strict[,3] == 1) & (dt.strict[,6] == 1) & (dt.strict[,5] == 0)))
downDeplGenes2 = names(which((dt.strict[,3] == -1) & (dt.strict[,6] == -1) & (dt.strict[,5] == 0)))
notUpDeplGenes2 = names(which((dt.strict[,3] == 0) & (dt.strict[,6] == 0) & (dt.strict[,5] == 1)))
notDownDeplGenes2 = names(which((dt.strict[,3] == 0) & (dt.strict[,6] == 0) & (dt.strict[,5] == -1)))

In [ ]:
notUpDeplGenes = intersect(notUpDeplGenes1, notUpDeplGenes2)
notDownDeplGenes = intersect(notDownDeplGenes1, notDownDeplGenes2)
upDeplGenes = intersect(upDeplGenes1, upDeplGenes2)
downDeplGenes = intersect(downDeplGenes1, downDeplGenes2)

In [ ]:
length(notUpDeplGenes)
length(notDownDeplGenes)
length(upDeplGenes)
length(downDeplGenes)

In [ ]:
for (i in v$genes$SYMBOL[which(v$genes$ENTREZID %in% downDeplGenes)]){
    cat(paste0(i, "\n"))
}
for (i in v$genes$SYMBOL[which(v$genes$ENTREZID %in% notUpDeplGenes)]){
    cat(paste0(i, "\n"))
}
write.csv(file = "../RANKL_effect_reduced.csv", row.names = F,
          x = c(v$genes$SYMBOL[which(v$genes$ENTREZID %in% downDeplGenes)], v$genes$SYMBOL[which(v$genes$ENTREZID %in% notUpDeplGenes)]))

In [ ]:
for (i in v$genes$SYMBOL[which(v$genes$ENTREZID %in% upDeplGenes)]){
    cat(paste0(i, "\n"))
}
for (i in v$genes$SYMBOL[which(v$genes$ENTREZID %in% notDownDeplGenes)]){
    cat(paste0(i, "\n"))
}
write.csv(file = "../RANKL_effect_increased.csv", row.names = F,
         x = c(v$genes$SYMBOL[which(v$genes$ENTREZID %in% upDeplGenes)], v$genes$SYMBOL[which(v$genes$ENTREZID %in% notDownDeplGenes)]))

Enrich links:  
[upregulated](http://amp.pharm.mssm.edu/Enrichr/enrich?dataset=3v95x)  
[downregulated](http://amp.pharm.mssm.edu/Enrichr/enrich?dataset=3v96c)

In [ ]:
print(which(apply(dt.multi[,c(7,9,10)]!=0, 1, all)))
dt.multi[(which(apply(dt.multi[,c(7,9,10)]!=0, 1, all))),]
# Gene 65112 (Pmepa1, prostate transmembrane protein) is up by arg depletion and down by recArg1

### Differences between MCSF only and MCSF + RANKL

In [ ]:
ranklOnlyGenes = intersect(names(which((dt.strict[,3] != 0))), names(which((dt.multi[,3] != 0))))
length(ranklOnlyGenes)

In [ ]:
for (i in v$genes$SYMBOL[which(v$genes$ENTREZID %in% ranklOnlyGenes)]){
    cat(paste0(i, "\n"))
}

In [ ]:
search_kegg_organism('mmu', by='kegg_code')

In [ ]:
pdf("../enrichment_kegg.pdf")
enrichRanklOnly <- as.data.frame(enrichKEGG(gene = ranklOnlyGenes, organism = 'mmu', pvalueCutoff = 1, qvalueCutoff = 0.1))
enrichRanklOnly$intergenes <- sapply(enrichRanklOnly$GeneRatio, function(x) as.numeric(strsplit(x, "/")[[1]][1]))
enrichRanklOnly$setgenes <- sapply(enrichRanklOnly$BgRatio, function(x) as.numeric(strsplit(x, "/")[[1]][1]))
enrichRanklOnly$p.adjust <- -log10(enrichRanklOnly$p.adjust)
# 8254 genes are part of the Kegg annotations, including 214 genes differentially expressed upon RANKL addition 
attach(enrichRanklOnly)
enrichRanklOnly$oddsratio <- (intergenes*(8254-(214 - intergenes)))/((214 - intergenes)*(setgenes - intergenes)) 
detach(enrichRanklOnly)
attach(enrichRanklOnly)
enrichRanklOnly$labs <- ifelse(oddsratio>4,Description,'')
colpal = colorRampPalette(c("#000000", "#FF0000"))(max(intergenes))
ggplot(enrichRanklOnly, aes(y=oddsratio, x=p.adjust, color = intergenes, label = labs)) +
  scale_colour_gradientn(colours = colpal, limits=c(0,max(intergenes)),
  breaks=c(0,round(max(max(intergenes))/2),max(max(intergenes))), name = "Shared genes") + 
  geom_point(size = 5) + theme_light() + theme(text=element_text(size=26, family="sans"), 
  panel.grid.minor=element_blank(), legend.position = c(0.8, 0.175), legend.spacing = unit(10, "cm"),
  legend.background = element_rect(size=0.3, color = "black"), legend.margin=margin(5,5,10,5)) +  
  xlab("-log10(corrected p-value)") + ylab("Odds-ratio") + ggtitle("KEGG enrichment with RANKL") + 
  geom_text_repel(size = 6.5, point.padding = 0.3, force = 33.511, segment.alpha = 0.5)
detach(enrichRanklOnly)
dev.off()

In [ ]:
sessionInfo()